<a href="https://colab.research.google.com/github/KalyaniTBhandwalkar/Assignment/blob/main/Readability_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
import re
import os.path
import requests
from bs4 import BeautifulSoup
import pandas as pd
import string
#import spacy
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.tag import pos_tag_sents
nltk.download('punkt')
from nltk import sent_tokenize
nltk.download('vader_lexicon') #required for Sentiment Analysis
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Unzip the files 

In [ ]:
from zipfile import ZipFile

with ZipFile('/content/StopWords.zip', 'r') as f:

#extract in different directory
  f.extractall('/content/')
from zipfile import ZipFile

with ZipFile('/content/MasterDictionary.zip', 'r') as f:

#extract in different directory
  f.extractall('/content/')

# Preprocess File

In [ ]:

#Count Syllable
def count_sy(word):
    return len(
        re.findall('(?!e$)[aeiouy]+', word, re.I) +
        re.findall('^[^aeiouy]*e$', word, re.I)
        )

#Preprocess the file
def file_process(filename):
  
# load data
  file = open(filename, 'rt',encoding = "ISO-8859-1")
  text = file.read()
  Content =text
  
  sen_len = 0 #sentence Counter
  word_len = 0  #word Counter
  num_charc = 0 #character Counter
  for line in Content:
     words = line.split()
     sen_len += line.count(".")
     word_len += len(words)
     num_charc += len(line.replace(' ', ''))

  file.close()
   #syllable count if syllable in words >=2
  syllable=0   
  words = text.split()
  word_len=len(words)   
  for w in words:
    syllable_count=count_sy(w)
    if(syllable_count>=2):
      syllable=syllable+1
  
  # count pronouns
  pattern='(I|we|my|ours|(?-i:us))'
  result = re.findall(pattern, text)
  pronouns=len(result)
  text = text.lower()
  word_len=len(words)
  
  # remove punctuation from each word
  table = str.maketrans('', '', string.punctuation)
  stripped = [w.translate(table) for w in words]
  word_count=len(stripped)
  tokens = [w.lower() for w in stripped]
  words = [word for word in stripped if word.isalpha()]
  #Stemming
  porter = PorterStemmer()
  words = [porter.stem(word) for word in words]
  #remove Stopwords
  stop_words = stopwords.words('english')
  stemmed = [w for w in words if not w in stop_words]
  return stemmed,sen_len,word_len,syllable,word_count,syllable_count,pronouns,num_charc

# Clean text

In [ ]:

def cleanTxt(fi):
  st1,word_Counter,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(("/content/StopWords/StopWords_Auditor.txt"))
  fi = [i for i in fi if i not in st1]
  st1,word_Counter,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(("/content/StopWords/StopWords_Currencies.txt"))
  fi = [i for i in fi if i not in st1]
  st1,word_Counter,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(("/content/StopWords/StopWords_DatesandNumbers.txt"))
  fi = [i for i in fi if i not in st1]
  st1,word_Counter,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(("/content/StopWords/StopWords_Generic.txt"))
  fi = [i for i in fi if i not in st1]
  st1,word_Counter,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(("/content/StopWords/StopWords_GenericLong.txt"))
  fi = [i for i in fi if i not in st1]
  st1,word_Counter,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(("/content/StopWords/StopWords_Geographic.txt"))
  fi = [i for i in fi if i not in st1]
  st1,word_Counter,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(("/content/StopWords/StopWords_Names.txt"))
  fi = [i for i in fi if i not in st1]
  return fi

# Main program

In [ ]:

save_path = '/content/'
data=pd.read_excel("/content/Input.xlsx")

headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:60.0) Gecko/20100101 Firefox/60.0"
}

# the target we want to open	
url=data['URL']
id=data['URL_ID']
#Create empty dataframe
df = pd.DataFrame(data=None, index=None, columns=['URL_ID','URL','POSITIVE SCORE','NEGATIVE SCORE','POLARITY SCORE','SUBJECTIVITY SCORE','AVG SENTENCE LENGTH','PERCENTAGE OF COMPLEX WORDS','FOG INDEX','AVG NUMBER OF WORDS PER SENTENCE','COMPLEX WORD COUNT','WORD COUNT','SYLLABLE PER WORD','PERSONAL PRONOUNS','AVG WORD LENGTH'])
#open with GET method
for link in url:
  resp = requests.get(link, headers=headers)
  soup = BeautifulSoup(resp.content, 'html.parser')
  content = resp.content
  print("===========================")
  title=soup.title.string
  l=soup.find('div',class_='td-post-content')
  url_id=data.loc[data['URL']==link,['URL_ID']]
  url_id=url_id['URL_ID'].values[0]
  url_id=url_id.astype(int)
  print("Successfully opened the web page ",url_id)
  completeName="{}{}{}".format(save_path, url_id,".txt")
  f= open(completeName, 'w+')
  f.write(title)
  f.write(l.text)
  f.close() 
  #Open the targeted file
  fi,sen_len,word_len,syllable,word_count,syllable_count,pronouns,num_charc=file_process(completeName)
  #Clean text by removing stopwords
  stop_word_removed=cleanTxt(fi)
  Total_Words_after_cleaning=len(stop_word_removed)
  print("===========================")
  
  #positive_word
  pos1=open("/content/MasterDictionary/positive-words.txt","r",encoding = "ISO-8859-1")
  pos= pos1.read()
  pos=pos.split()
  porter = PorterStemmer()
  pos = [porter.stem(word) for word in pos]
  stop_words = stopwords.words('english')
  pos = [w for w in pos if not w in stop_words]
  positive_word = [i for i in stop_word_removed if i  in pos]
  Positive_Score=len(positive_word)
  #Negative words
  neg1 = open("/content/MasterDictionary/negative-words.txt","r",encoding = "ISO-8859-1")
  neg=neg1.read()
  
  neg=neg.split()
  porter = PorterStemmer()
  neg = [porter.stem(word) for word in neg]
  stop_words = stopwords.words('english')
  neg = [w for w in neg if not w in stop_words]
  negative_word=[i for i in stop_word_removed if i  in neg]
  Negative_Score=len(negative_word)

  #Polarity Score
  pol_score = (Positive_Score-Negative_Score )/ ((Positive_Score  + Negative_Score ) + 0.000001)  
  #Subjectivity Score
  sub_score = (Positive_Score+Negative_Score )/((Total_Words_after_cleaning) + 0.000001) 
  #Average Number of Words Per Sentence = the total number of words / the total number of sentences
  #Average Sentence Length 
  ACL= word_len/ sen_len  
  #Percentage of Complex words 
  PCW= syllable / word_len
  #Fog index                                                          
  fog_index = 0.4 * (ACL + PCW)
  #AVG WORD LENGTH
  AWL  =num_charc/word_len
  #new row value
  df2 = {'URL_ID': url_id,
          'URL' :link,
          'POSITIVE SCORE' : Positive_Score,
          'NEGATIVE SCORE': Negative_Score,
          'POLARITY SCORE':pol_score ,
          'SUBJECTIVITY SCORE':sub_score ,
           'AVG SENTENCE LENGTH': ACL ,
           'PERCENTAGE OF COMPLEX WORDS':PCW ,
           'FOG INDEX': fog_index,
           'AVG NUMBER OF WORDS PER SENTENCE':ACL ,
           'COMPLEX WORD COUNT':syllable ,
           'WORD COUNT':word_count ,
           'SYLLABLE PER WORD':syllable_count ,
           'PERSONAL PRONOUNS':pronouns ,
           'AVG WORD LENGTH':AWL
         }
  # append the row to dataframe
  df = df.append(df2, ignore_index = True)


Successfully opened the web page  1
Successfully opened the web page  2
Successfully opened the web page  3
Successfully opened the web page  4
Successfully opened the web page  5
Successfully opened the web page  6
Successfully opened the web page  7
Successfully opened the web page  8
Successfully opened the web page  9
Successfully opened the web page  10
Successfully opened the web page  11
Successfully opened the web page  12
Successfully opened the web page  13
Successfully opened the web page  14
Successfully opened the web page  15
Successfully opened the web page  16
Successfully opened the web page  17
Successfully opened the web page  18
Successfully opened the web page  19
Successfully opened the web page  20
Successfully opened the web page  21
Successfully opened the web page  22
Successfully opened the web page  23
Successfully opened the web page  24
Successfully opened the web page  25
Successfully opened the web page  26
Successfully opened the web page  27
Successful

In [ ]:
#dataframe first 5 rows

df.head()

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/is-telehealth...,52,9,0.704918,0.204698,26.037037,0.479374,10.606564,26.037037,337,703,4,25,5.620199
1,2,https://insights.blackcoffer.com/how-telehealt...,68,15,0.638554,0.256966,17.690909,0.404933,7.238337,17.690909,394,973,5,44,5.154162
2,3,https://insights.blackcoffer.com/is-telemedici...,159,36,0.630769,0.240148,19.206522,0.506508,7.885212,19.206522,895,1767,4,68,5.810413
3,4,https://insights.blackcoffer.com/is-telehealth...,149,24,0.722543,0.202576,17.330097,0.516527,7.138649,17.330097,922,1785,3,83,5.938375
4,5,https://insights.blackcoffer.com/how-people-di...,149,45,0.536082,0.246193,22.691358,0.452666,9.257610,22.691358,832,1838,1,90,5.722524


# Save result dataframe to excel file

In [ ]:
OutputfileName="{}{}{}".format(save_path, "Output Data Structure",".xlsx")
df.to_excel(OutputfileName, index = False)